# Feature Selection & Feature Engineering

This notebook will cover fundamental methods used to develop ML algorithms, such as preparing data for a ML model (feature engineering).

**Topics:**


1.   Feature Selection
2.   Feature Engineering
3.   Preparing Data



**Goals:**


1.   Understand how to select important features for a ML algorithm
2.   Understand how to clean data for a ML algorithm
3.   Become familiar with common methods for preparing data to train a ML model



## Import Packages

The first thing we do at the beginning of any script.



1.   **Pandas:** Working with datasets. Arguably the most widely-used data-science Python package.
2.   **NumPy:** Scientific computing package for working with vectors & matrices. 
3. **MatplotLib:** Tool for dataset vizualizations.
4. **Sci-Kit Learn:** Open-source ML algorithms.
5. **Stats Models:** Open-source scientific computing package for statistical operations.

In [20]:
from statsmodels.formula.api import ols
from sklearn.model_selection import train_test_split
import pandas as pd, numpy as np, matplotlib.pyplot as plt
from sklearn.preprocessing import LabelEncoder, StandardScaler
from sklearn.impute import SimpleImputer

## Feature Selection & Feature Engineering

In this section we will cover how to prepare a dataset for a ML model.

Note - it is called data "science" for a reason. Each use-case can benefit from different methods and implementations, it is the job of the data scientist to experiment with these options and choose the best one. 

Here we will be going over common methods utilized for feature selection & engineering.

#### **Feature Selection**

Feature selection is the process of selecting and removing features from the dataset, so that the ML model can learn in an optimal manner.

**Read-in Dataset**

In [36]:
data = pd.read_csv('https://raw.githubusercontent.com/j0sephsasson/Pepsi-Training-Course/main/datasets/heart.csv')
data.head()

,age,sex,cp,trtbps,chol,fbs,restecg,thalachh,exng,oldpeak,slp,caa,thall,output
0,63,1,3,145,233,1,0,150,0,2.3,0,0,1,1
1,37,1,2,130,250,0,1,187,0,3.5,0,0,2,1
2,41,0,1,130,204,0,0,172,0,1.4,2,0,2,1
3,56,1,1,120,236,0,1,178,0,0.8,2,0,2,1
4,57,0,0,120,354,0,1,163,1,0.6,2,0,2,1


**Split data into X & y. (features & target)**

In [37]:
# Split into features/target
all_columns = list(data.columns)
target = all_columns.pop()
features = all_columns 

print(target)
print(features)

X = data[features]
y = data[target]

output
['age', 'sex', 'cp', 'trtbps', 'chol', 'fbs', 'restecg', 'thalachh', 'exng', 'oldpeak', 'slp', 'caa', 'thall']


**Examine null values in each feature**

In [10]:
# Check for null values
X.isnull().sum()

age         0
sex         0
cp          0
trtbps      0
chol        0
fbs         0
restecg     0
thalachh    0
exng        0
oldpeak     0
slp         0
caa         0
thall       0
dtype: int64

**Examine Cardinality.** 

This is an important concept - how many unique values in a particular feature?

This is called *Cardinality*

In [11]:
# Check cardinality
def show_cardinality(frame):
  for col in frame.columns:
     cardinality = len(frame[col].unique())
     print(col)
     print(cardinality)
     print()

show_cardinality(X)

age
41

sex
2

cp
4

trtbps
49

chol
152

fbs
2

restecg
3

thalachh
91

exng
2

oldpeak
40

slp
3

caa
5

thall
4



**Perform Feature Selection**

Now we will define a function to remove features that have any one of the following attributes to be true:


*   Cardinality above 85% (extremely high amount of unique values)
*   Cardinality below 0.1% (basically every value is the same)
*   Null value count above 60% (most values are null)



In [16]:
def feature_selection(data, features):
    """
    This function drops features with high cardinality

    Params:
      -- 'data': pd.DataFrame
      -- 'features': list[str]
      
    Returns:
      -- list of selected features 
    """

    good_features = []

    for f in features:
      cardinality = len(data[f].unique())
      null_count = data[f].isnull().sum()

      if cardinality > np.percentile(range(0,len(data)),85):
        continue
      elif cardinality < np.percentile(range(0,len(data)),0.1):
        continue
      elif null_count > np.percentile(range(0,len(data)),60):
        continue
      else:
        good_features.append(f)

    return good_features

good_features = feature_selection(X, features)
print('Removed Features: ', [i for i in features if i not in good_features])
print(good_features)

Removed Features:  []
['age', 'sex', 'cp', 'trtbps', 'chol', 'fbs', 'restecg', 'thalachh', 'exng', 'oldpeak', 'slp', 'caa', 'thall']


#### **Feature Engineering**

Feature engineering is the process of cleaning selected features for the model.

We want to scale & normalize numerical data, as well as numerically encode categorical features so they can be normalized & scaled.

**Common steps (in order):**


1.   Impute null values
2.   Numerically encode categorical features
3.   Scale & normalize numeric features (they are all numbers at this point)





**1.   Impute Null Values**





*   For numerical data we will replace null values with the mean of the feature
*   For categorical data we will replace the null values with the most freqeuent value

These are very basic ways to impute a feature. Much more complex methods exist.



In [25]:
# initialize a SimpleImputer with strategy 'mean'
imputer = SimpleImputer(strategy='mean') # used with numerical features

test = pd.Series([1,2,2,4,5,6, np.nan, np.nan, np.nan, 7,8,9])
test_imputed = imputer.fit_transform(test.values.reshape(-1, 1))

print('BEFORE:')
print(test.values)

print()

print('AFTER:')
print(test_imputed)

BEFORE:
[ 1.  2.  2.  4.  5.  6. nan nan nan  7.  8.  9.]

AFTER:
[[1.        ]
 [2.        ]
 [2.        ]
 [4.        ]
 [5.        ]
 [6.        ]
 [4.88888889]
 [4.88888889]
 [4.88888889]
 [7.        ]
 [8.        ]
 [9.        ]]




**2.   Encode Categorical Data**





*   Each unique value is assigned a number 1 through the length of unique values



In [29]:
# initialize a LabelEncoder
encoder = LabelEncoder()

# initialize imputer with strategy 'most_frequent'
imputer = SimpleImputer(strategy='most_frequent') # for categorical features

test = pd.Series(['yessir','hello', 'goodbye', 'hello', 'pancakes',
                  np.nan, np.nan, np.nan])

# impute null vals
test_imputed = imputer.fit_transform(test.values.reshape(-1, 1))

# encode the imputed, categorical data
test_encoded = encoder.fit_transform(test_imputed.reshape(test_imputed.shape[0],))

print('BEFORE:')
print(test.values)

print()

print('AFTER:')
print(test_encoded)

BEFORE:
['yessir' 'hello' 'goodbye' 'hello' 'pancakes' nan nan nan]

AFTER:
[3 1 0 1 2 1 1 1]





**3.   Scale & Normalize Data**





*   Will normalize a feature so the standard deviation is 1 and the mean is 0. This is also known as a 'Z-Score'
*   Each new value tells us how many standard deviations the original value was from the mean





In [31]:
# initialize scaler
scaler = StandardScaler()

# turn encoded data back into pandas format
series = pd.Series(test_encoded)

# scale data
series = scaler.fit_transform(series.values.reshape(-1, 1))

print('ORIGINAL:')
print(test_encoded)

print()

print('SCALED:')
print(series)

ORIGINAL:
[3 1 0 1 2 1 1 1]

SCALED:
[[ 2.11057941]
 [-0.30151134]
 [-1.50755672]
 [-0.30151134]
 [ 0.90453403]
 [-0.30151134]
 [-0.30151134]
 [-0.30151134]]


**Perform Feature Engineering**

Now we will define a function that will:


1.   Impute numerical values with the mean of that feature
2.   Impute categorical features with the most frequent value
3.   Encode categorical features to be numeric
4.   Scale & normalize the remaining data



In [38]:
def clean_dataset(frame, features):
  """
    This function performs feature engineering on a dataframe

    1. Imputing
    2. Encoding
    3. Scaling

    Params:
      -- 'data': pd.DataFrame
      -- 'features': list[str]
      
    Returns:
      -- 'data': pd.DataFrame
    """

  data = frame.copy()

  # Perform imputing
  for col in features:

    # if we have category use most frequent
    if data[col].dtypes == 'O':
        imputer = SimpleImputer(strategy='most_frequent')
        data[col] = imputer.fit_transform(data[col].values.reshape(-1, 1))

    # if we have number use mean
    elif data[col].dtypes == 'int64' or data[col].dtypes == 'float64':
        imputer = SimpleImputer(strategy='mean')
        data[col] = imputer.fit_transform(data[col].values.reshape(-1, 1))

  ## encode categorical features ##
  for col in features:
    if data[col].dtypes == 'O':
      data[col] = LabelEncoder().fit_transform(data[col])

  ## scale numercial features ##
  for col in features:
    if data[col].dtypes == 'int64' or data[col].dtypes == 'float64':
      data[col] = StandardScaler().fit_transform(data[col].values.reshape(-1, 1))

  return data

X_clean = clean_dataset(frame=X, features=features)

In [39]:
print('ORIGINAL')
print()
X.head()

ORIGINAL



,age,sex,cp,trtbps,chol,fbs,restecg,thalachh,exng,oldpeak,slp,caa,thall
0,63,1,3,145,233,1,0,150,0,2.3,0,0,1
1,37,1,2,130,250,0,1,187,0,3.5,0,0,2
2,41,0,1,130,204,0,0,172,0,1.4,2,0,2
3,56,1,1,120,236,0,1,178,0,0.8,2,0,2
4,57,0,0,120,354,0,1,163,1,0.6,2,0,2


In [40]:
print('CLEANED')
print()
X_clean.head()

CLEANED



,age,sex,cp,trtbps,chol,fbs,restecg,thalachh,exng,oldpeak,slp,caa,thall
0,0.952197,0.681005,1.973123,0.763956,-0.256334,2.394438,-1.005832,0.015443,-0.696631,1.087338,-2.274579,-0.714429,-2.148873
1,-1.915313,0.681005,1.002577,-0.092738,0.072199,-0.417635,0.898962,1.633471,-0.696631,2.122573,-2.274579,-0.714429,-0.512922
2,-1.474158,-1.468418,0.032031,-0.092738,-0.816773,-0.417635,-1.005832,0.977514,-0.696631,0.310912,0.976352,-0.714429,-0.512922
3,0.180175,0.681005,0.032031,-0.663867,-0.198357,-0.417635,0.898962,1.239897,-0.696631,-0.206705,0.976352,-0.714429,-0.512922
4,0.290464,-1.468418,-0.938515,-0.663867,2.082050,-0.417635,0.898962,0.583939,1.435481,-0.379244,0.976352,-0.714429,-0.512922
